In [1]:
# encoding=utf-8


import time
import random
import logging

import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score

from generate_dataset import *



class SVM(object):

    def __init__(self, kernel='linear',epsilon = 0.001):
        self.kernel = kernel
        self.epsilon = epsilon

    def _init_parameters(self, features, labels):
        '''
        初始化一些参数
        '''
        self.X = features
        self.Y = labels

        self.b = 0.0
        self.n = len(features[0])
        self.N = len(features)
        self.alpha = [0.0] * self.N
        self.E = [self._E_(i) for i in range(self.N)]

        self.C = 1000
        self.Max_Interation = 5000


    def _satisfy_KKT(self, i):
        ygx = self.Y[i] * self._g_(i)
        if abs(self.alpha[i])<self.epsilon:
            return ygx > 1 or ygx == 1
        elif abs(self.alpha[i]-self.C)<self.epsilon:
            return ygx < 1 or ygx == 1
        else:
            return abs(ygx-1) < self.epsilon

    def is_stop(self):
        for i in range(self.N):
            satisfy = self._satisfy_KKT(i)

            if not satisfy:
                return False
        return True

    def _select_two_parameters(self):
        '''
        按照书上7.4.2选择两个变量
        '''
        index_list = [i for i in range(self.N)]

        i1_list_1 = filter(lambda i: self.alpha[i] > 0 and self.alpha[i] < self.C, index_list)
        i1_list_2 = list(set(index_list) - set(i1_list_1))

        i1_list = list(i1_list_1)
        i1_list.extend(i1_list_2)

        for i in i1_list:
            if self._satisfy_KKT(i):
                continue

            E1 = self.E[i]
            max_ = (0, 0)

            for j in index_list:
                if i == j:
                    continue

                E2 = self.E[j]
                if abs(E1 - E2) > max_[0]:
                    max_ = (abs(E1 - E2), j)

            return i, max_[1]

    def _K_(self, x1, x2):
        '''
        核函数
        '''

        if self.kernel == 'linear':
            return sum([x1[k] * x2[k] for k in range(self.n)])
        if self.kernel == 'poly':
            return (sum([x1[k] * x2[k] for k in range(self.n)])+1)**3



        print('没有定义核函数')
        return 0

    def _g_(self, i):
        '''
        公式(7.104)
        '''
        result = self.b

        for j in range(self.N):
            result += self.alpha[j] * self.Y[j] * self._K_(self.X[i], self.X[j])

        return result

    def _E_(self, i):
        '''
        公式(7.105)
        '''
        return self._g_(i) - self.Y[i]

    def try_E(self,i):
        result = self.b-self.Y[i]
        for j in range(self.N):
            if self.alpha[j]<0 or self.alpha[j]>self.C:
                continue
            result += self.Y[j]*self.alpha[j]*self._K_(self.X[i],self.X[j])
        return result


    def train(self, features, labels):

        self._init_parameters(features, labels)

        for times in range(self.Max_Interation):
            # if self.is_stop():
            #     return

            logging.debug('iterater %d' % times)

            i1, i2 = self._select_two_parameters()

            L = max(0, self.alpha[i2] - self.alpha[i1])
            H = min(self.C, self.C + self.alpha[i2] - self.alpha[i1])

            if self.Y[i1] == self.Y[i2]:
                L = max(0, self.alpha[i2] + self.alpha[i1] - self.C)
                H = min(self.C, self.alpha[i2] + self.alpha[i1])

            E1 = self.E[i1]
            E2 = self.E[i2]
            eta = self._K_(self.X[i1], self.X[i1]) + self._K_(self.X[i2], self.X[i2]) - 2 * self._K_(self.X[i1], self.X[i2])     # 公式(7.107)

            alpha2_new_unc = self.alpha[i2] + self.Y[i2] * (E1 - E2) / eta        # 公式(7.106)

            # 公式(7.108)
            alph2_new = 0
            if alpha2_new_unc > H:
                alph2_new = H
            elif alpha2_new_unc < L:
                alph2_new = L
            else:
                alph2_new = alpha2_new_unc

            # 公式(7.109)
            alph1_new = self.alpha[i1] + self.Y[i1] * \
                self.Y[i2] * (self.alpha[i2] - alph2_new)

            # 公式(7.115) 及 公式(7.116)
            b_new = 0
            b1_new = -E1 - self.Y[i1] * self._K_(self.X[i1], self.X[i1]) * (alph1_new - self.alpha[i1]) - self.Y[i2] * self._K_(self.X[i2], self.X[i1]) * (alph2_new - self.alpha[i2]) + self.b
            b2_new = -E2 - self.Y[i1] * self._K_(self.X[i1], self.X[i2]) * (alph1_new - self.alpha[i1]) - self.Y[i2] * self._K_(self.X[i2], self.X[i2]) * (alph2_new - self.alpha[i2]) + self.b

            if alph1_new > 0 and alph1_new < self.C:
                b_new = b1_new
            elif alph2_new > 0 and alph2_new < self.C:
                b_new = b2_new
            else:
                b_new = (b1_new + b2_new) / 2

            self.alpha[i1] = alph1_new
            self.alpha[i2] = alph2_new
            self.b = b_new

            self.E[i1] = self._E_(i1)
            self.E[i2] = self._E_(i2)


    def _predict_(self,feature):
        result = self.b

        for i in range(self.N):
            result += self.alpha[i]*self.Y[i]*self._K_(feature,self.X[i])

        if result > 0:
            return 1
        return -1

    def predict(self,features):
        results = []

        for feature in features:
            results.append(self._predict_(feature))

        return results


if __name__ == "__main__":
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)

    print('Start read data')

    time_1 = time.time()

    # 选取 2/3 数据作为训练集， 1/3 数据作为测试集
    train_features, train_labels, test_features, test_labels = generate_dataset(2000,visualization=False)

    time_2 = time.time()
    print('read data cost ',time_2 - time_1,' second','\n')

    print('Start training')
    svm = SVM()
    svm.train(train_features, train_labels)

    time_3 = time.time()
    print('training cost ',time_3 - time_2,' second','\n')

    print('Start predicting')
    test_predict = svm.predict(test_features)
    time_4 = time.time()
    print('predicting cost ',time_4 - time_3,' second','\n')

    score = accuracy_score(test_labels,test_predict)
    print("svm1 the accruacy socre is ", score)



C:\ProgramData\Anaconda3\envs\python3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Start read data
read data cost  0.012000799179077148  second 

Start training


DEBUG:root:iterater 0
DEBUG:root:iterater 1
DEBUG:root:iterater 2
DEBUG:root:iterater 3
DEBUG:root:iterater 4
DEBUG:root:iterater 5
DEBUG:root:iterater 6
DEBUG:root:iterater 7
DEBUG:root:iterater 8
DEBUG:root:iterater 9
DEBUG:root:iterater 10
DEBUG:root:iterater 11
DEBUG:root:iterater 12
DEBUG:root:iterater 13
DEBUG:root:iterater 14
DEBUG:root:iterater 15
DEBUG:root:iterater 16
DEBUG:root:iterater 17
DEBUG:root:iterater 18
DEBUG:root:iterater 19
DEBUG:root:iterater 20
DEBUG:root:iterater 21
DEBUG:root:iterater 22
DEBUG:root:iterater 23
DEBUG:root:iterater 24
DEBUG:root:iterater 25
DEBUG:root:iterater 26
DEBUG:root:iterater 27
DEBUG:root:iterater 28
DEBUG:root:iterater 29
DEBUG:root:iterater 30
DEBUG:root:iterater 31
DEBUG:root:iterater 32
DEBUG:root:iterater 33
DEBUG:root:iterater 34
DEBUG:root:iterater 35
DEBUG:root:iterater 36
DEBUG:root:iterater 37
DEBUG:root:iterater 38
DEBUG:root:iterater 39
DEBUG:root:iterater 40
DEBUG:root:iterater 41
DEBUG:root:iterater 42
DEBUG:root:iterater 4

DEBUG:root:iterater 346
DEBUG:root:iterater 347
DEBUG:root:iterater 348
DEBUG:root:iterater 349
DEBUG:root:iterater 350
DEBUG:root:iterater 351
DEBUG:root:iterater 352
DEBUG:root:iterater 353
DEBUG:root:iterater 354
DEBUG:root:iterater 355
DEBUG:root:iterater 356
DEBUG:root:iterater 357
DEBUG:root:iterater 358
DEBUG:root:iterater 359
DEBUG:root:iterater 360
DEBUG:root:iterater 361
DEBUG:root:iterater 362
DEBUG:root:iterater 363
DEBUG:root:iterater 364
DEBUG:root:iterater 365
DEBUG:root:iterater 366
DEBUG:root:iterater 367
DEBUG:root:iterater 368
DEBUG:root:iterater 369
DEBUG:root:iterater 370
DEBUG:root:iterater 371
DEBUG:root:iterater 372
DEBUG:root:iterater 373
DEBUG:root:iterater 374
DEBUG:root:iterater 375
DEBUG:root:iterater 376
DEBUG:root:iterater 377
DEBUG:root:iterater 378
DEBUG:root:iterater 379
DEBUG:root:iterater 380
DEBUG:root:iterater 381
DEBUG:root:iterater 382
DEBUG:root:iterater 383
DEBUG:root:iterater 384
DEBUG:root:iterater 385
DEBUG:root:iterater 386
DEBUG:root:itera

DEBUG:root:iterater 688
DEBUG:root:iterater 689
DEBUG:root:iterater 690
DEBUG:root:iterater 691
DEBUG:root:iterater 692
DEBUG:root:iterater 693
DEBUG:root:iterater 694
DEBUG:root:iterater 695
DEBUG:root:iterater 696
DEBUG:root:iterater 697
DEBUG:root:iterater 698
DEBUG:root:iterater 699
DEBUG:root:iterater 700
DEBUG:root:iterater 701
DEBUG:root:iterater 702
DEBUG:root:iterater 703
DEBUG:root:iterater 704
DEBUG:root:iterater 705
DEBUG:root:iterater 706
DEBUG:root:iterater 707
DEBUG:root:iterater 708
DEBUG:root:iterater 709
DEBUG:root:iterater 710
DEBUG:root:iterater 711
DEBUG:root:iterater 712
DEBUG:root:iterater 713
DEBUG:root:iterater 714
DEBUG:root:iterater 715
DEBUG:root:iterater 716
DEBUG:root:iterater 717
DEBUG:root:iterater 718
DEBUG:root:iterater 719
DEBUG:root:iterater 720
DEBUG:root:iterater 721
DEBUG:root:iterater 722
DEBUG:root:iterater 723
DEBUG:root:iterater 724
DEBUG:root:iterater 725
DEBUG:root:iterater 726
DEBUG:root:iterater 727
DEBUG:root:iterater 728
DEBUG:root:itera

KeyboardInterrupt: 

In [29]:
len(svm.alpha)
svm.X[0]

[2.0594147018065012, 7.4583360805084702]